In [6]:
# %%
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='5'

import torch
import torchaudio
import scipy.io.wavfile as wf
import numpy as np
import base64
import hashlib
import json 
import time
import uuid
import requests
import scipy
from speechbrain.pretrained import EncoderClassifier
import random
import hmac
import json
from datetime import datetime
from time import mktime
from urllib.parse import urlencode
from wsgiref.handlers import format_date_time
#import talentedsoft
from pydub import AudioSegment
#import scikits.audiolab
##科大讯飞声纹识别 tjs's 
APPId = "fcfefe42"
APISecret = "NmE5NTZkNzFlMzg3MGM0ZDBmZTdkODYx"
APIKey = "aba4d1c2815577b411d5d5a4966796aa"
class Gen_req_url(object):
    """生成请求的url"""

    def sha256base64(self, data):
        sha256 = hashlib.sha256()
        sha256.update(data)
        digest = base64.b64encode(sha256.digest()).decode(encoding='utf-8')
        return digest

    def parse_url(self, requset_url):
        stidx = requset_url.index("://")
        host = requset_url[stidx + 3:]
        # self.schema = requset_url[:stidx + 3]
        edidx = host.index("/")
        if edidx <= 0:
            raise Exception("invalid request url:" + requset_url)
        self.path = host[edidx:]
        self.host = host[:edidx]

    # build websocket auth request url
    def assemble_ws_auth_url(self, requset_url, api_key, api_secret, method="GET"):
        self.parse_url(requset_url)
        now = datetime.now()
        date = format_date_time(mktime(now.timetuple()))
        # date = "Thu, 12 Dec 2019 01:57:27 GMT"
        signature_origin = "host: {}\ndate: {}\n{} {} HTTP/1.1".format(self.host, date, method, self.path)
        signature_sha = hmac.new(api_secret.encode('utf-8'), signature_origin.encode('utf-8'),
                                 digestmod=hashlib.sha256).digest()
        signature_sha = base64.b64encode(signature_sha).decode(encoding='utf-8')
        authorization_origin = "api_key=\"%s\", algorithm=\"%s\", headers=\"%s\", signature=\"%s\"" % (
            api_key, "hmac-sha256", "host date request-line", signature_sha)
        authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
        values = {
            "host": self.host,
            "date": date,
            "authorization": authorization
        }

        return requset_url + "?" + urlencode(values)


def gen_req_body(apiname, APPId, file_path=None):
    """
    生成请求的body
    :param apiname
    :param APPId: Appid
    :param file_name:  文件路径
    :return:
    """
    if apiname == 'createFeature':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "createFeature",
                    "groupId": "8",
                    "featureId": "cmu_scottish",
                    "featureInfo": "wav08",
                    "createFeatureRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    elif apiname == 'createGroup':

        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "createGroup",
                    "groupId": "8",
                    "groupName": "cmu_dataset",
                    "groupInfo": "cmu_dataset",
                    "createGroupRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            }
        }
   
    elif apiname == 'queryFeatureList':

        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "queryFeatureList",
                    "groupId": "8",
                    "queryFeatureListRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            }
        }
    elif apiname == 'searchFea':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "searchFea",
                    "groupId": "8",
                    "topK": 10,
                    "searchFeaRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    elif apiname == 'searchScoreFea':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "searchScoreFea",
                    "groupId": "iFLYTEK_examples_groupId",
                    "dstFeatureId": "p374",
                    "searchScoreFeaRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    else:
        raise Exception(
            "输入的apiname不在[createFeature, createGroup, deleteFeature, queryFeatureList, searchFea, searchScoreFea,updateFeature]内，请检查")
    return body

def req_url(api_name, APPId, APIKey, APISecret, file_path=None):
    """
    开始请求
    :param APPId: APPID
    :param APIKey:  APIKEY
    :param APISecret: APISecret
    :param file_path: body里的文件路径
    :return:
    """
    gen_req_url = Gen_req_url()
    body = gen_req_body(apiname=api_name, APPId=APPId, file_path=file_path)
    request_url = gen_req_url.assemble_ws_auth_url(requset_url='https://api.xf-yun.com/v1/private/s782b4996', method="POST", api_key=APIKey, api_secret=APISecret)
    headers = {'content-type': "application/json", 'host': 'api.xf-yun.com', 'appid': '$APPID'}
    response = requests.post(request_url, data=json.dumps(body), headers=headers)
    tempResult = json.loads(response.content.decode('utf-8'))
    #print(tempResult)
    #print(base64.b64decode(tempResult['payload'][api_name + 'Res']['text']))
"""
 * 1.声纹识别接口,请填写在讯飞开放平台-控制台-对应能力页面获取的APPID、APIKey、APISecret
 * 2.groupId要先创建,然后再在createFeature里使用,不然会报错23005,修改时需要注意保持统一
 * 3.音频base64编码后数据(不超过4M),音频格式需要16K、16BIT的MP3音频。
 * 4.主函数只提供调用示例,其他参数请到对应类去更改,以适应实际的应用场景。
"""
def req_score(api_name, APPId,  APIKey, APISecret, file_path=None):
    gen_req_url = Gen_req_url()
    body = gen_req_body(apiname=api_name, APPId=APPId, file_path=file_path)
    request_url = gen_req_url.assemble_ws_auth_url(requset_url='https://api.xf-yun.com/v1/private/s782b4996', method="POST", api_key=APIKey, api_secret=APISecret)
    headers = {'content-type': "application/json", 'host': 'api.xf-yun.com', 'appid': '$APPID'}
    response = requests.post(request_url, data=json.dumps(body), headers=headers)
    tempResult = json.loads(response.content.decode('utf-8'))
    #print(tempResult)
    return base64.b64decode(tempResult['payload'][api_name + 'Res']['text'])#,base64.b64decode(tempResult['payload'][api_name + 'Res']['text']['scoreList'][0]['score'])


def GetSpeakerScore(speaker,value):
    m = len(value)
    result = ''
    for i in range(0,m):
        result+=chr(value[i]) #switch to string
    result = eval(result)
    v = list(result.values())[0]
    #print(v)
    lenv=len(v)
    return v[0]['featureId'], v[0]['score'], v[1]['featureId'],v[1]['score']
#talentedsoft
import hashlib
import requests
URL = 'http://119.3.22.24:3997/'
url_lst = ['detectregister', 'registeruser', 'addsample', 'detectquery','trainmodel', 'deleteuser', 'verifymodel', 'identifymodel']
def get_str_md5(str):
    m = hashlib.md5()
    m.update(str.encode("utf8"))
    return m.hexdigest()
class TalentedSoft:
    def __init__(self,content,token,userid):
        self.body = {
            'userid': userid,
            'token': token,
            'content': content,
            'step': '0',
        }
        self.files = {
            'file': '0',
        }

    def url_req(self, userid, req, uploadfile):
        self.body['userid'] = userid
        if req == 2:
            for i in range(5):
                self.body['step'] = f'{i + 1}'
                self.files['file'] = open(uploadfile[i], 'rb')
                r = requests.post(URL + url_lst[req], data=self.body, files=self.files)
                print(r.text)
        elif (req == 6) | (req == 7):
            self.files['file'] = open(uploadfile, 'rb')
            r = requests.post(URL + url_lst[req], data=self.body, files=self.files)
            return r.text
        else:
            r = requests.post(URL + url_lst[req], data=self.body)
            print(r.text)

def talent_score(testfile,userid):#这个函数是我自己加的，可以用来获取得分,是进行1:n比对用的，若要进行1：1比对需要把req改成6，同时userid要填写被比对的人
    req = 7
    content = 'arctic_a0008.wav'
    token = get_str_md5('Chenpinji&CMU') #Chenpinji&Cpjkey是Vox数据集(应该是word里面那组)  Chenpinji&CMU是cmu数据集
    ts = TalentedSoft(content,token,userid)
    ans = ts.url_req(userid, req, testfile)
    #print(ans)
    return ans.split(",")[1].split("[")[1].split(" ")[0], float(ans.split(",")[4].split(":")[1][1:6])
    
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")
def dvector_extraction(wav_tensor1):
    transform = torchaudio.transforms.MFCC(sample_rate = sample_rate,n_mfcc = 40, melkwargs={"n_fft": 1024, "hop_length": 128, "n_mels": 64, "center": False},)
    mel_tensor1 = transform(wav_tensor1)[0].T
    emb_tensor1 = dvector.embed_utterance(mel_tensor1)
    return emb_tensor1
def xvector_extraction(wav_ts):
    return classifier.encode_batch(wavs=wav_ts)
def SNR(wav_tensor1, noise_tensor):
    temp1 = wav_tensor1
    temp2 = noise_tensor
    temp2 = temp2.detach().numpy()
    temp1 = temp1.numpy()
    ans = 20*np.log10(np.linalg.norm(temp1, ord=2)/np.linalg.norm(temp2, ord=2))
    return ans
def getSign(timestamp, nonce):
    hs = hashlib.sha256()
    appkey = "zvpcvm5hxib3jz3vt2jshqxjndcywwo2qnx7s6iy"
    secret = '3c8a61e2bdb0ee2d0af74814142ba2ee'
    hs.update((appkey + timestamp + secret + nonce).encode('utf-8'))
    signature = hs.hexdigest().upper()
    return signature
def identifyFeatureByGroupId(confirmFeatureFileName):
    identify_feature = open(confirmFeatureFileName, 'rb').read()
    # 声纹base64字符串
    audio_data = base64.b64encode(identify_feature)
    timestamp = str(int(time.time() * 1000))
    nonce = str(uuid.uuid1()).replace('-', '')
    sign = getSign(timestamp, nonce)
    headers = {"Content-Type": "application/json"}
    appkey = "zvpcvm5hxib3jz3vt2jshqxjndcywwo2qnx7s6iy"
    groupId = '15'
    host = 'https://ai-vpr.hivoice.cn'
    identifyFeatureByGroupIdEndPoint = '/vpr/v1/identifyFeatureByGroupId'
    identify_feature_param = {
        "appkey": appkey,
        "timestamp": timestamp,
        "nonce": nonce,
        "sign": sign,
        "groupId": groupId,
        "topN": 10,
        "audioData": audio_data.decode(),
        "audioSampleRate": 16000,
        "audioFormat": "mp3"
    }
    #print('identify_feature_param', identify_feature_param)
    identify_feature_resp = requests.post(url=host + identifyFeatureByGroupIdEndPoint,
                                          data=json.dumps(identify_feature_param),
                                          headers=headers)
    identify_feature_result = json.loads(identify_feature_resp.content)
    return identify_feature_result['data'][0]['featureInfo'], identify_feature_result['data'][0]['score'],identify_feature_result['data'][1]['featureInfo'], identify_feature_result['data'][1]['score']
def loss_function(AdvTensor, TarTensor,noise_tensor):
    return torch.norm(AdvTensor - TarTensor)# +  0.15 * torch.norm(noise_tensor, float('inf'))
def trans_wav_mp3(file_path):
    """
    将wav文件转为mp3文件，保存到cmu_mp3
    """
    sourcefile = AudioSegment.from_wav(file_path)
    #filename = file_path.split('/')[-1].split('.wav')[0].replace(' ', '_') + '.mp3'
    filename = file_path.split('/')[-1].split('.wav')[0] + '.mp3'
    save_path = "/mnt/data/Chenpinji/xvectorAudio/mp3/"
    #print(filename)
    sourcefile.export(save_path + filename, format="mp3")

In [8]:
persons = ['cmu2']
#persons = ['id11015/i84R2Tpczqc/']
ofiles = ['0033']
#ofiles = ['00001']
for person in persons:
    for ofile in ofiles:
        owavname = 'arctic_a'+ ofile +'.wav' #cmu数据集
        filename1 = '/mnt/data/Chenpinji/cmu_dataset/'+ person+ '/wav/'+ owavname #cmu数据集
        #owavname = ofile +'.wav'
        #filename1 = "/mnt/data/voxData/wav/" + person + owavname #vox数据集
        print()
        print(filename1)
        filename2 = '/mnt/data/Chenpinji/cmu_dataset/cmu6/wav/arctic_a0008.wav'
        wav_tensor1, sample_rate = torchaudio.load(filename1,normalize = False)
        wav_tensor2, sample_rate = torchaudio.load(filename2,normalize = False)
        noise_tensor = torch.normal(mean=80.,std=50.,size=(1,len(wav_tensor1[0])))
        wav_tensor1 = wav_tensor1.to(torch.float)
        wav_tensor2 = wav_tensor2.to(torch.float)
        noise_tensor.to(device=device)
        noise_tensor.requires_grad = True
        optimizer = torch.optim.Adam([noise_tensor],lr = 3)
        # epochs = random.randint(500,1000)
        epochs = 2000
        for epoch in range(epochs):
            AdvTensor = xvector_extraction(wav_tensor1+noise_tensor)
            TarTensor = xvector_extraction(wav_tensor2)
            loss = loss_function(AdvTensor, TarTensor, noise_tensor)
        #     print("epoch={} loss={}".format(epoch, loss))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if epoch % 50 == 0 or epoch == epochs - 1:
                print("epoch={} loss={}".format(epoch, loss))
                print("SNR:",SNR(wav_tensor1, noise_tensor))
                oname = 'xGen_'+str(epoch)
                audio_name = '/mnt/data/Chenpinji/xvectorAudio/mp3/'+ oname +'.wav'
                temp = wav_tensor1 + noise_tensor 
                temp = temp.detach().numpy()
                scaled = np.array(np.clip(np.round(temp[0]),-2 ** 15, 2 ** 15 - 1), dtype = np.int16)
                #print("epoch={} loss={}".format(epoch, loss))
                wf.write(audio_name, 16000 ,scaled)
                trans_wav_mp3(audio_name)
                #value = req_score(api_name='searchFea', APPId=APPId,APIKey=APIKey, APISecret=APISecret, file_path='/mnt/data/Chenpinji/xvectorAudio/mp3/'+ oname + '.mp3')
                #speaker1,score1,speaker2,score2 = GetSpeakerScore('cmu7',value)
                file_path='/mnt/data/Chenpinji/xvectorAudio/mp3/'+ oname + '.mp3'
                #speaker1,score1,speaker2,score2 = identifyFeatureByGroupId(file_path)
                asperson, s = talent_score(audio_name,'cmu7')
                print('classified as:',asperson,'Score is:',s)
                #print('second place is:',speaker2, 'Score is:',score2)
                # speaker1,score1,speaker2,score2 = identifyFeatureByGroupId(audio_name)
                #print('classified as:',speaker1,'Score is:',score1)
                #print('second place is:',speaker2, 'Score is:',score2)
                #print()


/mnt/data/Chenpinji/cmu_dataset/cmu2/wav/arctic_a0033.wav
epoch=0 loss=145.28965759277344
SNR: 31.912202835083008
classified as: cmu2 Score is: 0.996
epoch=50 loss=56.911075592041016
SNR: 29.746394157409668
classified as: cmu2 Score is: 0.984
epoch=100 loss=35.09718704223633
SNR: 27.905547618865967
classified as: cmu2 Score is: 0.971
epoch=150 loss=25.365795135498047
SNR: 26.574127674102783
classified as: cmu2 Score is: 0.963
epoch=200 loss=19.836593627929688
SNR: 25.551018714904785
classified as: cmu2 Score is: 0.936
epoch=250 loss=16.118179321289062
SNR: 24.726741313934326
classified as: cmu2 Score is: 0.917
epoch=300 loss=13.469961166381836
SNR: 24.054312705993652
classified as: cmu2 Score is: 0.903
epoch=350 loss=11.598688125610352
SNR: 23.477649688720703
classified as: cmu2 Score is: 0.883
epoch=400 loss=10.158514976501465
SNR: 22.99529790878296
classified as: cmu2 Score is: 0.876
epoch=450 loss=9.181899070739746
SNR: 22.600889205932617
classified as: cmu2 Score is: 0.868
epoch=5